In [1]:
import pandas as pd
import numpy as np
import hashlib

In [2]:
# Load the dataset from both csv and json, compare that they are equal and loaded correctly
df = pd.read_csv("ocr-reduced.csv")
from_json = pd.read_json("ocr-reduced.jsonl", lines=True)
df.equals(pd.json_normalize(from_json["HS_DATA"])[df.columns].replace("", np.nan))

True

In [3]:
# Create hashing function
def hash(x):
    return hashlib.sha256(x.encode()).hexdigest()

In [4]:
# Apply hashing function
df["name_of_covered_entity"] = df["name_of_covered_entity"].apply(hash)
for breach in from_json["HS_DATA"]:
    breach["name_of_covered_entity"] = hash(breach["name_of_covered_entity"])
from_json[["HS_RECORD_UUID", "HS_SOURCE", "HS_UUID"]] = from_json[
    ["HS_RECORD_UUID", "HS_SOURCE", "HS_UUID"]
].applymap(hash)

In [5]:
# Verify that they are still equal
df.equals(pd.json_normalize(from_json["HS_DATA"])[df.columns].replace("", np.nan))

True

In [6]:
# Save files
df.to_csv("ocr-reduced.csv", index=False)
from_json.to_json("ocr-reduced.jsonl", orient="records", lines=True)